In [3]:
# import libraries
import cv2
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

# Function to detect keypoints and compute descriptors
def detect_and_compute(detector, descriptor, img):
    keypoints = detector.detect(img, None)
    keypoints, descriptors = descriptor.compute(img, keypoints)
    return keypoints, descriptors

# Function to read images from a directory
def read_images(query_image_path, train_images_dir):
    query_img = cv2.imread(query_image_path, cv2.IMREAD_GRAYSCALE)
    train_images = [cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) for img_path in glob.glob(os.path.join(train_images_dir, "*"))]
    return query_img, train_images

# Function to match descriptors and return the matches
def match_descriptors(descriptor_extractor, query_descriptors, train_descriptors):
    # Match descriptors.
    matches = descriptor_extractor.knnMatch(query_descriptors, train_descriptors, k=2)
    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good_matches.append(m)
    return good_matches

# Function to draw matches between images
def draw_matches(img1, kp1, img2, kp2, matches):
    # Convert images to color so that we can draw colored lines
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2RGB)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2RGB)
    
    # Create a blank image with enough space to hold both input images
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    output_img = np.zeros((max(h1, h2), w1 + w2, 3), dtype="uint8")
    
    # Place the first image to the left
    output_img[:h1, :w1] = img1
    # Place the next image to the right of it
    output_img[:h2, w1:] = img2
    
    # For each pair of matching keypoints, draw a line between them
    for match in matches:
        # Get the matching keypoints
        pt1 = tuple(map(int, kp1[match.queryIdx].pt))
        pt2 = tuple(map(int, kp2[match.trainIdx].pt))
        pt2 = tuple([pt2[0] + w1, pt2[1]])  # offset the x-coordinate of keypoints in the second image by the width of the first image
        
        # Draw a line between the matching keypoints (this is why we needed to convert the images to color)
        cv2.line(output_img, pt1, pt2, (0, 255, 0), 1)
    
    return output_img

def main(query_image_path, train_images_dir):
    # Create SIFT detector and descriptor
    sift = cv2.SIFT_create()
    
    # Read images
    query_img, train_images = read_images(query_image_path, train_images_dir)
    
    # Detect keypoints and compute descriptors for query image
    query_keypoints, query_descriptors = detect_and_compute(sift, sift, query_img)
    
    # Detect keypoints and compute descriptors for train images
    train_keypoints_descriptors = [detect_and_compute(sift, sift, img) for img in train_images]

    # Initialize a list to keep track of which keypoints appear in all images
    matches_in_all_images = set(range(len(query_keypoints)))

    # Initialize a list to store the matches for each train image
    all_matches = []

    # Match descriptors and save result images
    for i, (keypoints, descriptors) in enumerate(train_keypoints_descriptors):
        matches = match_descriptors(cv2.BFMatcher(cv2.NORM_L2), query_descriptors, descriptors)
        all_matches.append(matches)

        # Update the set of keypoints that appear in all images
        matches_in_all_images &= set(m.queryIdx for m in matches)

    # Filter the matches to include only those that appear in all images
    all_matches = [[m for m in matches if m.queryIdx in matches_in_all_images] for matches in all_matches]

    # Print the number of matches from image 1 to each other image separately
    for i, matches in enumerate(all_matches, start=2):
        print(f'Number of matches from image 1 to image {i}: {len(matches)}')

    # Draw the matches
    # Create a blank image with enough space to hold all images
    h1, w1 = query_img.shape[:2]
    h2, w2 = max(img.shape for img in train_images)
    output_img = np.zeros((2 * h1, 3 * w2, 3), dtype="uint8")

    # Place the first image at the center of the first row
    output_img[:h1, w2:2*w2, :] = cv2.cvtColor(query_img, cv2.COLOR_GRAY2BGR)

    # For each train image, draw the matches to the right of the previous image
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]  # red, green, and blue
    for i, (matches, (keypoints, _)) in enumerate(zip(all_matches, train_keypoints_descriptors)):
        x_offset = i * w2  # offset x-coordinate by the width of the previous images in the same row
        y_offset = h1  # offset y-coordinate by the height of the first row
        output_img[y_offset:y_offset+h2, x_offset:x_offset+w2, :] = cv2.cvtColor(train_images[i], cv2.COLOR_GRAY2BGR)
        for match in matches:
            pt1 = tuple(map(int, query_keypoints[match.queryIdx].pt))
            pt1 = (w2 + pt1[0], pt1[1])  # offset x-coordinate by the width of one image
            pt2 = tuple(map(int, keypoints[match.trainIdx].pt))
            pt2 = (pt2[0] + x_offset, pt2[1] + y_offset)  # offset x-coordinate and y-coordinate by the offsets of the current train image
            cv2.line(output_img, pt1, pt2, colors[i], 2)  # draw a line with a thickness of 2 and a color corresponding to the current image
    
    # Save result image
    cv2.imwrite('result_centeralized.png', output_img)



# Usage
main('query/image1.png', 'train/')


Number of matches from image 1 to image 2: 10
Number of matches from image 1 to image 3: 10
Number of matches from image 1 to image 4: 10
